# Get focal length/image width

In [6]:
from PIL import Image
import exifread

# Load the image
image_path = "kb60.jpg"  # Replace with your image file path
image = Image.open(image_path)

# Get image width
image_width = image.width

# Open the image file for reading EXIF data
with open(image_path, 'rb') as img_file:
    tags = exifread.process_file(img_file)
    focal_length_raw = tags.get('EXIF FocalLength', 'Not available')
    focal_length_35mm_raw = tags.get('EXIF FocalLengthIn35mmFilm', 'Not available')
    
    # Convert focal length to float
    if focal_length_raw != 'Not available':
        focal_length = float(focal_length_raw.values[0])
    else:
        focal_length = 'Not available'
    
    # Convert 35mm focal length to int
    if focal_length_35mm_raw != 'Not available':
        focal_length_35mm = int(focal_length_35mm_raw.values[0])
    else:
        focal_length_35mm = 'Not available'

print(f"Image Width: {image_width}")
print(f"Focal Length: {focal_length} mm")
print(f"Focal Length (35mm equivalent): {focal_length_35mm} mm")

Image Width: 5712
Focal Length: 5.960000038146973 mm
Focal Length (35mm equivalent): 26 mm


In [7]:
import math

def estimate_sensor_width(focal_length_mm, focal_length_35mm):
    """
    Estimate sensor width (mm) using crop factor.
    - crop_factor = focal_35mm / focal_mm
    - sensor_width = 36 mm / crop_factor
    """
    if not focal_length_mm or not focal_length_35mm:
        return None
    crop_factor = focal_length_35mm / focal_length_mm
    return 36.0 / crop_factor


sensor_width_mm = estimate_sensor_width(focal_length, focal_length_35mm)
print(f"Estimated sensor width: {sensor_width_mm:.2f} mm")

Estimated sensor width: 8.25 mm


In [8]:
def calculate_object_size(pixel_length, image_width_pixels, sensor_width_mm, focal_length_mm, distance_mm):
    """
    Calculate real-world object size from pixel measurements.
    
    Formula:
    1. pixel_size_mm = sensor_width_mm / image_width_pixels
    2. object_size_on_sensor_mm = pixel_length × pixel_size_mm
    3. real_object_size_mm = (object_size_on_sensor_mm × distance_mm) / focal_length_mm
    
    Args:
        pixel_length: Length of object in pixels
        image_width_pixels: Image width in pixels
        sensor_width_mm: Camera sensor width in mm
        focal_length_mm: Focal length in mm
        distance_mm: Distance from camera to object in mm
    
    Returns:
        Real object size in mm
    """
    # Calculate pixel size in mm
    pixel_size_mm = sensor_width_mm / image_width_pixels
    
    # Calculate object size on sensor
    object_size_on_sensor_mm = pixel_length * pixel_size_mm
    
    # Calculate real object size using similar triangles
    real_object_size_mm = (object_size_on_sensor_mm * distance_mm) / focal_length_mm
    
    return real_object_size_mm

# Your data
image_width_pixels = image_width
focal_length_mm = focal_length
sensor_width_mm = sensor_width_mm
pixel_length = 2267

# You need to know the distance to the object
distance_mm = 600  # Example: 1000mm = 1 meter (you need to measure this!)

object_size_mm = calculate_object_size(
    pixel_length=pixel_length,
    image_width_pixels=image_width_pixels,
    sensor_width_mm=sensor_width_mm,
    focal_length_mm=focal_length_mm,
    distance_mm=distance_mm
)

print(f"Real object size: {object_size_mm:.2f} mm ({object_size_mm/10:.2f} cm)")

Real object size: 329.72 mm (32.97 cm)
